In [1]:
import pandas as pd

# SCHEMI, ANNOTAZIONI, GS

In [6]:
url = "https://raw.githubusercontent.com/GConte47/tirocinio/master/df%20omap/files/SCHEMI.csv"

SCHEMI = pd.read_csv(url, delimiter=",")  # Cambia il delimitatore se non è ";"

url = "https://raw.githubusercontent.com/GConte47/tirocinio/refs/heads/master/df%20omap/gold_standard.csv"

GS = pd.read_csv(url, delimiter= ",")


url = "https://raw.githubusercontent.com/GConte47/tirocinio/refs/heads/master/df%20omap/files/medcat/Medcat_SNOMED_Df_CS_08_ANC.csv"
EsempioAnnotazione = pd.read_csv(url, delimiter=",")

In [2]:
#versione locale
SCHEMI = pd.read_csv('files/SCHEMI.csv', delimiter=",")
GS = pd.read_csv('files/gold_standard.csv', delimiter=",")
EsempioAnnotazione = pd.read_csv('files/medcat/Medcat_SNOMED_Df_CS_08_ANC.csv', delimiter=",")

In [3]:
# controllo coerenza

SCHEMI.columns, GS.columns

(Index(['SOURCE', 'AttributoNomeCompleto', 'DescrizioneCompleta',
        'DescrizioneTabella', 'DescrizioneAttributo', 'Tabella', 'Attributo'],
       dtype='object'),
 Index(['TipoDiAnnotazione', 'Ontologies', 'Source', 'AttributoNomeCompleto',
        'entity_id', 'pretty_name', 'cui'],
       dtype='object'))

In [4]:
# occorre   usare gli stessi attributi nomi per lo stesso concetto
GS.rename(columns={"Source": "SOURCE"}, inplace=True)


In [5]:
# in GS 'SOURCE', 'AttributoNomeCompleto' deve essere una foreign key riferita a SCHEMI
len(GS), len(pd.merge(GS,SCHEMI))

(321, 180)

In [6]:
diff = GS[['SOURCE', 'AttributoNomeCompleto']].merge(
    SCHEMI[['SOURCE', 'AttributoNomeCompleto']],
    on=['SOURCE', 'AttributoNomeCompleto'],
    how='left',
    indicator=True
)

# Seleziona solo le coppie presenti in GS ma non in SCHEMI
not_in_schemi = diff[diff['_merge'] == 'left_only'].drop(columns=['_merge'])

not_in_schemi

,SOURCE,AttributoNomeCompleto
0,OMOP,location-location
1,OMOP,location-location
2,OMOP,location-location
3,OMOP,location-location
4,OMOP,location-location
...,...,...
251,OMOP,person-datetime_of_birth
252,OMOP,person-datetime_of_death
253,OMOP,person-datetime_of_death
254,OMOP,person-datetime_of_death


In [7]:
diff = GS[['SOURCE', 'AttributoNomeCompleto']].merge(
    SCHEMI[['SOURCE', 'AttributoNomeCompleto']],
    on=[ 'AttributoNomeCompleto'],
    how='left',
    indicator=True
)

# Seleziona solo le coppie presenti in GS ma non in SCHEMI
diff[diff['_merge'] == 'left_only'].drop(columns=['_merge'])

,SOURCE_x,AttributoNomeCompleto,SOURCE_y
0,OMOP,location-location,NaN
1,OMOP,location-location,NaN
2,OMOP,location-location,NaN
3,OMOP,location-location,NaN
4,OMOP,location-location,NaN
...,...,...,...
251,OMOP,person-datetime_of_birth,NaN
252,OMOP,person-datetime_of_death,NaN
253,OMOP,person-datetime_of_death,NaN
254,OMOP,person-datetime_of_death,NaN


In [8]:
# vediamo perchè mancano
SCHEMI[SCHEMI['AttributoNomeCompleto'].str.contains('location', na=False, case=False)]

,SOURCE,AttributoNomeCompleto,DescrizioneCompleta,DescrizioneTabella,DescrizioneAttributo,Tabella,Attributo
9,OMOP,person-location_id,the person domain contains records that unique...,the person domain contains records that unique...,a foreign key to the place of residency for th...,person,location_id
127,OMOP,location-location_id,the location table represents a generic way to...,the location table represents a generic way to...,a unique identifier for each geographic location.,location,location_id
128,OMOP,location-address_1,the location table represents a generic way to...,the location table represents a generic way to...,"the address field 1, typically used for the st...",location,address_1
129,OMOP,location-address_2,the location table represents a generic way to...,the location table represents a generic way to...,"the address field 2, typically used for additi...",location,address_2
130,OMOP,location-city,the location table represents a generic way to...,the location table represents a generic way to...,the city field as it appears in the source data.,location,city
131,OMOP,location-state,the location table represents a generic way to...,the location table represents a generic way to...,the state field as it appears in the source data.,location,state
132,OMOP,location-zip,the location table represents a generic way to...,the location table represents a generic way to...,the zip or postal code.,location,zip
133,OMOP,location-county,the location table represents a generic way to...,the location table represents a generic way to...,the county.,location,county
134,OMOP,location-location_source_value,the location table represents a generic way to...,the location table represents a generic way to...,the verbatim information that is used to uniqu...,location,location_source_value
240,OMOP,care_site-location_id,the care_site table contains a list of uniquel...,the care_site table contains a list of uniquel...,a foreign key to the geographic location in th...,care_site,location_id


In [21]:
# location-location 	 dovrebbe corrispondere a location-location_id
# RISPOSTA= location-location in realtà dovrebbe corrispondere solo a location dato che i codici sono corrispondenti alla tabella principale.

# location-address 	possiamo dire che corrisponde a location-address_1
#RISPOSTA= no, corrisponde a location address_1 e address_2. è un errore mio che li ho omessi per sbaglio dal gold standard (PROBLEMA RISOLTO)

# location-location_source-value 	 c'è anche in SCHEMI location-location_source_value, forse c'è qualche carattere strano nella stringa
# vedi con chatGPT come controllare
# anche qui un mio errore di battitura. l'errore è che anzichè mettere '_' ho messo '-' (PROBLEMA RISOLTO)


In [ ]:
### 1) da fare : dopo aver sistemato questi casi
### riprendere il notebook     tirocinio/df omap/PerControllare.ipynb
### per controllare di quanti elementi abbiamo il relativo gold standard


In [ ]:
### ora consideriamo le annotazioni

In [9]:
# controllo coerenza per EsempioAnnotazione
EsempioAnnotazione.columns

Index(['TipoDiAnnotazione', 'ontologies', 'SOURCE', 'AttributoNomeCompleto',
       'entity_id', 'pretty_name', 'cui', 'type_ids', 'types', 'source_value',
       'detected_name', 'acc', 'context_similarity'],
      dtype='object')

In [10]:
# in EsempioAnnotazione 'SOURCE', 'AttributoNomeCompleto' deve essere una foreign key riferita a SCHEMI
len(EsempioAnnotazione), len(pd.merge(EsempioAnnotazione,SCHEMI))

(5031, 5031)

In [ ]:
# ok

In [22]:
### METTERE TUTTE LE ANNOTAZIONI FATTE in un unico file
### e ripetere il controllo precedente
### ora ovviamente stiamo considerando tutti gli schemi non solo OMOP








In [26]:
#### Leggere il file delle annotazioni rispetto a BioPOrtal che avevo fatto io
AnnotazioniConBioportal=pd.read_csv("http://dbgroup.ing.unimore.it/OMAP/AnnotazioniConBioportal.csv")
# controllo coerenza con schemi
len(AnnotazioniConBioportal), len(pd.merge(AnnotazioniConBioportal,SCHEMI))

(12694, 12694)

In [27]:
# avevo considerato solo OMOP
AnnotazioniConBioportal.groupby('SOURCE').agg(
    num_distinct_AttributoNomeCompleto=('AttributoNomeCompleto', 'nunique'),
    num_distinct_TipoDiAnnotazione=('TipoDiAnnotazione', 'nunique'),
    num_distinct_Ontologia=('Ontologia', 'nunique'),
    num_distinct_Codice=('Codice', 'nunique'),
    mean_codice_per_attributo=('Codice', lambda x: x.nunique() / x.groupby(AnnotazioniConBioportal['AttributoNomeCompleto']).ngroup().nunique())
)


,num_distinct_AttributoNomeCompleto,num_distinct_TipoDiAnnotazione,num_distinct_Ontologia,num_distinct_Codice,mean_codice_per_attributo
SOURCE,,,,,
OMOP,267,4,2,604,2.262172


In [28]:
AnnotazioniConBioportal.columns

Index(['TipoDiAnnotazione', 'Ontologia', 'SOURCE', 'AttributoNomeCompleto',
       'NumProg', 'Elemento', 'Codice', 'Description'],
      dtype='object')

In [16]:
annotazioniMedcat = pd.read_csv('files/annotazioni.csv', delimiter=",")
annotazioniMedcat.columns

Index(['TipoDiAnnotazione', 'ontologies', 'SOURCE', 'AttributoNomeCompleto',
       'entity_id', 'pretty_name', 'cui'],
      dtype='object')

In [14]:
## per mettere assieme tutto e confrontarle occorre uniformare i nomi di questi attributi
## con quelle delle tue annotazioni di medcat

EsempioAnnotazione.columns





Index(['TipoDiAnnotazione', 'ontologies', 'SOURCE', 'AttributoNomeCompleto',
       'entity_id', 'pretty_name', 'cui', 'type_ids', 'types', 'source_value',
       'detected_name', 'acc', 'context_similarity'],
      dtype='object')

In [31]:
## controlla e dicidi tu quali nomi tenere
## a me sembra che
## entity_id --> è il numero progressivo  NumProg per 'TipoDiAnnotazione', 'Ontologia', 'SOURCE', 'AttributoNomeCompleto',

## cui --> Codice  sono i codici dei concetti

## pretty_name --> Description  sono i nomi dei concetti

## 'source_value' --> Elemento   ??

## altri elementi delle annotazioni di medcat ('acc', 'context_similarity', ...) non hanno corrispodenza in Bioportal e quindi per queste assumeranno valore nullo 

In [32]:
EsempioAnnotazione

,TipoDiAnnotazione,ontologies,SOURCE,AttributoNomeCompleto,entity_id,pretty_name,cui,type_ids,types,source_value,detected_name,acc,context_similarity
0,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Csm,beneficiarysummary-bene_birth_dt,1,Date of birth,184099003,['2680757'],['observable entity'],date of birth,date~of~birth,1.00,1.00
1,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Csm,beneficiarysummary-bene_death_dt,1,Date of death,399753006,['2680757'],['observable entity'],date of death,date~of~death,1.00,1.00
2,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Csm,beneficiarysummary-bene_esrd_ind,2,End-stage renal disease,46177005,['9090192'],['disorder'],end stage renal disease,end~stage~renal~disease,1.00,1.00
3,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Csm,beneficiarysummary-sp_state_code,0,State,398070004,['75168589'],['environment'],state,state,1.00,1.00
4,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Csm,beneficiarysummary-bene_hi_cvrage_tot_mons,0,Total,255619001,['7882689'],['qualifier value'],total,total,0.99,0.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5026,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Synthea,providers-state,5,Abbreviation,398223008,['7882689'],['qualifier value'],abbreviation,abbreviation,1.00,1.00
5027,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Synthea,providers-zip,0,Patient,116154003,['31601201'],['person'],patient,patient,1.00,1.00
5028,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Synthea,providers-zip,2,Street address,398099009,['2680757'],['observable entity'],street address,street~address,1.00,1.00
5029,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Synthea,providers-utilization,0,Patient,116154003,['31601201'],['person'],patient,patient,1.00,1.00


DOMANDA: Il dataframe importato delle annotazioni di bioportal devo modificare i nomi delle colonne e unirlo con quello delle annotazioni giusto?


In [29]:
AnnotazioniConBioportal.rename(columns={"NumProg": "entity_id"}, inplace=True)


In [32]:
AnnotazioniConBioportal.rename(columns={"Ontologia": "ontologies"}, inplace=True)

In [35]:
AnnotazioniConBioportal.rename(columns={"Codice": "cui"}, inplace=True)

In [36]:
AnnotazioniConBioportal.rename(columns={"Description": "pretty_name"}, inplace=True)

In [38]:
AnnotazioniConBioportal.drop(columns=["Elemento"], inplace=True)

In [41]:
nuovo_ordine = ['TipoDiAnnotazione', 'ontologies', 'SOURCE', 'AttributoNomeCompleto',
       'entity_id', 'pretty_name', 'cui']

In [42]:
AnnotazioniConBioportal = AnnotazioniConBioportal[nuovo_ordine]


In [43]:
AnnotazioniConBioportal

,TipoDiAnnotazione,ontologies,SOURCE,AttributoNomeCompleto,entity_id,pretty_name,cui
0,BioportalAttributoNomeCompleto,LOINC,OMOP,person-person_id,1,Person,LP345156-6
1,BioportalAttributoNomeCompleto,LOINC,OMOP,person-person_id,2,Person,LP133774-2
2,BioportalAttributoNomeCompleto,LOINC,OMOP,person-person_id,3,Person,MTHU066373
3,BioportalAttributoNomeCompleto,LOINC,OMOP,person-person_id,4,Person:ID:Pt:^Patient:Nom,106193-6
4,BioportalAttributoNomeCompleto,LOINC,OMOP,person-person_id,5,Person,MTHU041262
...,...,...,...,...,...,...,...
12689,BioportalAttributo,SNOMEDCT,OMOP,provider-gender_source_value,4,Source (property),734842000
12690,BioportalAttributo,SNOMEDCT,OMOP,provider-gender_source_value,5,Source,260753009
12691,BioportalAttributo,SNOMEDCT,OMOP,provider-gender_source_concept_id,3,Gender,263495000
12692,BioportalAttributo,SNOMEDCT,OMOP,provider-gender_source_concept_id,4,Source (property),734842000


In [44]:
annotazioniMedcat

,TipoDiAnnotazione,ontologies,SOURCE,AttributoNomeCompleto,entity_id,pretty_name,cui
0,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Csm,beneficiarysummary-bene_birth_dt,1,Date of birth,184099003
1,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Csm,beneficiarysummary-bene_death_dt,1,Date of death,399753006
2,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Csm,beneficiarysummary-bene_esrd_ind,2,End-stage renal disease,46177005
3,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Csm,beneficiarysummary-sp_state_code,0,State,398070004
4,Medcat_SNOMED_Df_CS_08_ANC,SNOMED-CT,Csm,beneficiarysummary-bene_hi_cvrage_tot_mons,0,Total,255619001
...,...,...,...,...,...,...,...
25403,Medcat_UMLS_Df_Full_ANC,UMLS,Synthea,providers-utilization,3,patient care,C0017313
25404,Medcat_UMLS_Df_Full_ANC,UMLS,Synthea,providers-utilization,5,Numbers,C0237753
25405,Medcat_UMLS_Df_Full_ANC,UMLS,Synthea,providers-utilization,7,Encounter,C1947978
25406,Medcat_UMLS_Df_Full_ANC,UMLS,Synthea,providers-utilization,9,Performed By,C1550369


In [45]:
Annotazioni_full= pd.concat([annotazioniMedcat, AnnotazioniConBioportal], ignore_index=True)

In [47]:
Annotazioni_full.to_csv('files/annotazioni_full.csv', index=False)

appunto= ho notato che in bioportal le ontologie sono in snomed e loinc